In [1]:
import numpy as np
import os
import sys
import torch
import pickle
import re
from collections import OrderedDict
sys.path.append('./network/')
from networks import BranchedNetwork

In [6]:
#Load saved weights and biases- split into three .npy files to upload to github
weights_dir = os.getcwd() + '/network/weights/'
with open(weights_dir + 'network_weights_early_layers.p', 'rb') as f:
    weights_biases = pickle.load(f)
with open(weights_dir + 'network_weights_genre_branch.p', 'rb') as f:
    genre_branch = pickle.load(f)
with open(weights_dir + 'network_weights_word_branch.p', 'rb') as f:
    word_branch = pickle.load(f)

In [7]:
bn = BranchedNetwork()
keys = bn.state_dict().keys()

In [8]:
regex = re.compile('(\w+_branch)\.(\w+)\.block\.(?:0\.)*(\w+)')

In [12]:
state_dict = OrderedDict()
for key in keys:
    key_groups = regex.match(key)
    branch = key_groups.group(1)
    layer = key_groups.group(2)
    struct = key_groups.group(3)
    struct = 'W' if 'weight' in struct else 'b'
    if layer.startswith('fc') and layer.endswith('W'):
        arr = word_branch[layer][struct]
    elif layer.startswith('fc') and layer.endswith('G'):
        arr = genre_branch[layer][struct]
    else:
        arr = weights_biases[layer][struct]
    arr = torch.tensor(arr)
    if len(arr.shape) == 4:
        arr = torch.permute(arr, (3, 2, 0, 1))
    elif len(arr.shape) == 2:
        arr = torch.permute(arr, (1, 0))
    state_dict[key] = arr

conv1
(9, 9, 1, 96)

conv1
(96,)

conv2
(5, 5, 96, 256)

conv2
(256,)

conv3
(3, 3, 256, 512)

conv3
(512,)

conv4_W
(3, 3, 512, 1024)

conv4_W
(1024,)

conv5_W
(3, 3, 1024, 512)

conv5_W
(512,)

fc6_W
(18432, 1024)

fc6_W
(1024,)

fctop_W
(1024, 589)

fctop_W
(589,)

conv1
(9, 9, 1, 96)

conv1
(96,)

conv2
(5, 5, 96, 256)

conv2
(256,)

conv3
(3, 3, 256, 512)

conv3
(512,)

conv4_G
(3, 3, 512, 1024)

conv4_G
(1024,)

conv5_G
(3, 3, 1024, 512)

conv5_G
(512,)

fc6_G
(18432, 1024)

fc6_G
(1024,)

fctop_G
(1024, 43)

fctop_G
(43,)



In [10]:
bn.load(state_dict)

In [11]:
torch.save(state_dict, 'branched_network_weights.pt')